In [1]:
!pip install nltk gensim pandas numpy

In [2]:
import pandas as pd
import nltk
import gensim
from gensim import corpora
from gensim.models import Phrases
from gensim.models.ldamodel import LdaModel
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ckred\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ckred\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ckred\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
train_data = pd.read_csv('train_data.csv')
paragraph_data = pd.read_csv("train_input_paragraph.csv")

In [4]:
train_data.head()

,Unnamed: 0,Theme,Paragraph,Question,Answer_possible,Answer_text,Answer_start
0,2,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,True,['2003'],[526]
1,6,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What album made her a worldwide known artist?,True,['Dangerously in Love'],[505]
2,7,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,Who managed the Destiny's Child group?,True,['Mathew Knowles'],[360]
3,8,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyoncé rise to fame?,True,['late 1990s'],[276]
4,9,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What role did Beyoncé have in Destiny's Child?,True,['lead singer'],[290]


In [5]:
paragraph_data.head()

,id,paragraph,theme
0,1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,Beyoncé
1,2,Following the disbandment of Destiny's Child i...,Beyoncé
2,3,"A self-described ""modern-day feminist"", Beyonc...",Beyoncé
3,4,"Beyoncé Giselle Knowles was born in Houston, T...",Beyoncé
4,5,Beyoncé attended St. Mary's Elementary School ...,Beyoncé


In [6]:
lemmatizer = WordNetLemmatizer()
tokenizer = RegexpTokenizer(r'\w+')

docs = []

for idx in range(len(paragraph_data)):
    paragraph_data['paragraph'][idx] = paragraph_data['paragraph'][idx].lower()
    docs.append(tokenizer.tokenize(paragraph_data['paragraph'][idx]))
    
docs = [[token for token in doc if len(token) > 1] for doc in docs]

C:\Users\ckred\AppData\Local\Temp\ipykernel_13624\1655334862.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  paragraph_data['paragraph'][idx] = paragraph_data['paragraph'][idx].lower()


In [7]:
paragraph_data.head()

,id,paragraph,theme
0,1,beyoncé giselle knowles-carter (/biːˈjɒnseɪ/ b...,Beyoncé
1,2,following the disbandment of destiny's child i...,Beyoncé
2,3,"a self-described ""modern-day feminist"", beyonc...",Beyoncé
3,4,"beyoncé giselle knowles was born in houston, t...",Beyoncé
4,5,beyoncé attended st. mary's elementary school ...,Beyoncé


In [8]:
docs_lemmatized = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]

In [9]:
bigram = Phrases(docs_lemmatized, min_count=5)
for idx in range(len(docs_lemmatized)):
    for token in bigram[docs_lemmatized[idx]]:
        if '_' in token:
            docs_lemmatized[idx].append(token)

In [10]:
dictionary = corpora.Dictionary(docs_lemmatized)

In [11]:
corpus = [dictionary.doc2bow(text) for text in docs_lemmatized]

In [12]:
question_data = pd.read_csv("train_input_question.csv")
question_data.head()

,id,question,theme
0,1,When did Beyonce leave Destiny's Child and bec...,Beyoncé
1,2,What album made her a worldwide known artist?,Beyoncé
2,3,Who managed the Destiny's Child group?,Beyoncé
3,4,When did Beyoncé rise to fame?,Beyoncé
4,5,What role did Beyoncé have in Destiny's Child?,Beyoncé


In [14]:
answer_data = pd.read_csv("train_ground_truth.csv")
for i in range(len(answer_data)):
    answer_data['answers'][i] = answer_data['answers'][i][2:-2]
answer_data.head()

C:\Users\ckred\AppData\Local\Temp\ipykernel_13624\1948923624.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  answer_data['answers'][i] = answer_data['answers'][i][2:-2]


,question_id,paragraph_id,answers
0,1,[1],2003
1,2,[1],Dangerously in Love
2,3,[1],Mathew Knowles
3,4,[1],late 1990s
4,5,[1],lead singer


In [15]:
from nltk.tokenize import word_tokenize
docs_qa = []

for idx in range(len(paragraph_data)):
    question = question_data['question'][idx].lower()
    answer = answer_data['answers'][idx].lower()
    question_tokens = [lemmatizer.lemmatize(token) for token in word_tokenize(question) if token not in stop_words]
    answer_tokens = [lemmatizer.lemmatize(token) for token in word_tokenize(answer) if token not in stop_words]
    docs_qa.append(question_tokens + answer_tokens)

bigram_qa = Phrases(docs_qa, min_count=5)
for idx in range(len(docs_qa)):
    for token in bigram_qa[docs_qa[idx]]:
        if '_' in token:
            docs_qa[idx].append(token)

dictionary_qa = corpora.Dictionary(docs_qa)

corpus_qa = [dictionary_qa.doc2bow(text) for text in docs_qa]

In [26]:
# ldamodel_pre = LdaModel(corpus_qa, num_topics=10, id2word = dictionary_qa, passes=2000)
ldamodel = LdaModel(corpus, num_topics=10, id2word = dictionary, passes=40)

In [27]:
ldamodel.update(corpus_qa, chunksize=16, iterations=1, eval_every=10, passes=50)

In [20]:
from gensim.models.ensemblelda import EnsembleLda
from gensim.test.utils import common_texts

common_dictionary = corpora.Dictionary(common_texts)
common_corpus = [common_dictionary.doc2bow(text) for text in common_texts]
# Set training parameters.
num_topics = 10
chunksize = 2000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

elda = EnsembleLda(corpus=corpus_qa, id2word=dictionary_qa, num_topics=10, num_models=4)

# elda.add_model(ldamodel_pre)
# elda.recluster()

elda.add_model(LdaModel(corpus, num_topics=10, id2word = dictionary, passes=20))
elda.recluster()

ValueError: target ttda dimensions do not match. Topics must be 16517 but was 70901 elements large

In [28]:
def get_topic(question, ldamodel):
    question_tokens = [lemmatizer.lemmatize(token) for token in word_tokenize(question.lower()) if token not in stop_words]
    question_bow = dictionary.doc2bow(question_tokens)
    topic_probs = ldamodel[question_bow]
    return max(topic_probs, key=lambda x: x[1])[0]

# Create a function to get the most likely answer for a question
def get_answer(question, ldamodel):
    question_topic = get_topic(question, ldamodel)
    topic_answers = [text for text, topic in zip(docs_lemmatized, ldamodel[corpus]) if topic[0][0] == question_topic]
    return max(topic_answers, key=lambda x: len(x))

In [30]:
for i in range(len(question_data)):
    question_data['question'][i] = question_data['question'][i].lower()
    answer = get_answer(question_data['question'][i], ldamodel)
    print("Question: ", question_data['question'][i])
    print("Answer: ", answer)

C:\Users\ckred\AppData\Local\Temp\ipykernel_13624\612108899.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  question_data['question'][i] = question_data['question'][i].lower()


ValueError: max() arg is an empty sequence